# Getting started with Tensorflow

This Tutorial is based on the official "Getting Started With TensorFlow" Tutorial on https://www.tensorflow.org/get_started/get_started

# Tensors

- The central unit of data in TensorFlow is the tensor.
- A tensor consists of a set of primitive values shaped into an array of any number of dimensions. 
- A tensors rank is its number of dimensions. 

    - 3 -------------------------------------->: A rank 0 tensor; this is a scalar with shape []
    - [1. ,2., 3.] --------------------------->: A rank 1 tensor; this is a vector with shape [3]
    - [[1., 2., 3.], [4., 5., 6.]] ----------->: A rank 2 tensor; a matrix with shape [2, 3]
    - [[[1., 2., 3.]], [[7., 8., 9.]]] ------->: A rank 3 tensor with shape [2, 1, 3]

In [10]:
import tensorflow as tf
import matplotlib.image as mpimg
from matplotlib import pyplot as plt

# The Computational Graph + tf.Session()

TensorFlow Core consists out of two discrete sections:
    - Building the computational graph
    - Running the computational graph
    
A computational graph is a series of TensorFlow operations arranged into a graph of nodes. Each node takes zero or more tensors as input and produces a tensor as an output. One type of node is a constant. A tensorFlow constant takes no inputs, and it outpus a value it stores internally. 

In [11]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


node 1 and node 2 are two floating point Tensors. If we print this tensors, we don't get their values. We need to evaluate them so that they produce 3.0 and 4.0. To actually evaluate the nodes, we must run the computational graph within a session. A session encapsulates the control and state of the TensorFlow runtime. 

In [12]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


The code above creates a Session object and then calles its run method.

In [13]:
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ",sess.run(node3))

node3:  Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3):  7.0


![Illustration of The Computational Graph](images/TensorBoard_visualisation.PNG)

The Picture above is a visualisation of the current graph

# Placeholders
The created graph is not very interesting because it always produces a constant result (here it is 7.0). To "feed" the graph with a variaty of different values we need to parameterize the graph so it accepts external inputs, known as placeholders. A placeholder is a promis to provide a value later.

In [14]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

The created graph can be seen like a function with two variables a and b

In [15]:
print(sess.run(adder_node, {a: 3, b:4.5}))
print(sess.run(adder_node, {a: [1,3], b: [2, 4]}))

7.5
[ 3.  7.]


The TensorBoard graph looks like this:

![Illustration of The Computational Graph](images/TensorBoard_visualisation2.PNG)

In [16]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b:4.5}))

22.5


![Illustration of The Computational Graph](images/TensorBoard_visualisation3.PNG)

The graph gets more complex.
- adder_node contains the old graph which just added a and b
- add_and_tripple multiplies the result of the old graph by y = 3

# Variables
The placeholders are the inputs of our graph. But to make the model trainable, we need to be able to modify the graph to get new outputs with the same inputs. to modify our graph, we introduce variables. Variables allow us to add trainable parameters to our graph. They are constructed with a type and initial value.

In [17]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

![Illustration of The Computational Graph](images/Linear_graph.PNG)

Constants like at the beginning are initialised when you call tf.constant. Their values can never change. Variables insted are NOT initialized when you call tf.Variable. To initialise all the variables in a TensorFlow program, you must explicitly call a special operation as follows:

In [18]:
init = tf.global_variables_initializer()
sess.run(init)

Init is a handle to the TensorFlow sub-graph that initialilzes the global variables. Unitl we call sess.run(init), the variables are NOT initialised. To evaluate our graph, we need to call the init operator before! Otherwise the used variables are not initialised and we get multiple errors

# Create a loss function

Since x is a placeholder, we can evaluate our graph for multiple x-values.

In [19]:
print(sess.run(linear_model, {x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


We've created a model, but we don't know how good it is yet. To evaluate the model on training data, we need a y placeholder to provide the desired values, and we need to write a loss function. A loss function measures how far apart the current model is from the provided data. 
- Loss function
We will use a standard loss model for linear regression. 
- linear_model - y creates a vector. Each element stands for the delta between the current model and the provided data.
- tf.square calculates the square of each vector element.
- tf.reduce_sum summs all the squared errors to create a single scalar that abstracts the overall error.   

In [20]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

23.66


As you can see, our model is 23.66 (What ever that means) away from our data. 
- [0.  0.3  0.6  0.9] 
- vs 
- [0. -1.0 -2.0 -3.0]

What we need to do is, we need to change our modelparamters / graph variables W and b. 

In [21]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

0.0


With the help of the operation tf.assign, we are able to change the already initialised variables W and b to -1 and 1. We run the session again and calculate the loss which is now 0.0
- NICE^^
We guessed the "perfect" values of W and b, but whole point of machine learning is to find the correct model parameters automatically. We will show how to do that in the next section.

![Illustration of The Computational Graph](images/Linear_graph.PNG)

# tf.train API -> Gradient descent

TensorFlow provides optimizers that slowly change each variable in order to minimize the loss function. The simplest optimizer is called gradient descent.
- Gradient descent modiefies each variable according to the magnitude(Größe) of the derivative of loss with respect to that variable.
TensorFlow does this with just one command.
- tf.gradients

In [22]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
  sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})

print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


The result is [-1] and [1]!! PERFECT Now we have done actual machine learning.

# Complete program

In [23]:
import numpy as np
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)
# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
# training data
x_train = [1,2,3,4]
y_train = [0,-1,-2,-3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x:x_train, y:y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x:x_train, y:y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11
